[View in Colaboratory](https://colab.research.google.com/github/peaelle42/DNN_jump_start/blob/master/JS03_Char_RNN.ipynb)

## Imports

In [1]:
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [2]:
!ls
!

datalab  log  ngrok  ngrok-stable-linux-amd64.zip


## Load Dataset

In [18]:

#path = get_file('shakespeare-sonnets.txt', origin='https://www.dropbox.com/s/3wuwuc2ef36tdi0/shakespeare_sonnets.txt')
!wget https://www.dropbox.com/s/3wuwuc2ef36tdi0/shakespeare_sonnets.txt

with io.open('shakespeare_sonnets.txt', encoding='utf-8') as f:
    text = f.read().lower()
print('corpus length:', len(text))


corpus length: 102939


In [19]:
!ls

datalab  ngrok			       shakespeare_sonnets.txt
log	 ngrok-stable-linux-amd64.zip  shakespeare_sonnets.txt.1


In [20]:
len(text)

102939

In [21]:
!ls

datalab  ngrok			       shakespeare_sonnets.txt
log	 ngrok-stable-linux-amd64.zip  shakespeare_sonnets.txt.1


In [22]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 48


In [23]:
# cut the text in semi-redundant sequences of maxlen characters
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

nb sequences: 34300


In [24]:
print(sentences[0])
print(sentences[1])
print(sentences[2])
print(sentences[3])

the sonnets

by william shakespeare

   
 sonnets

by william shakespeare

      
nnets

by william shakespeare

         
ts

by william shakespeare

            


In [25]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


In [26]:
# build the model: a single LSTM
print('Build model...')
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

Build model...


In [0]:
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Sampling some text from the model

In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [0]:
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [0]:
#callback

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [0]:
model.fit(x, y,
          batch_size=128,
          epochs=60,
          callbacks=[print_callback])

Epoch 1/60
34300/34300 [==============================] - 29s 858us/step - loss: 2.2973

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " master mistress of my passion,
  a woma"
 master mistress of my passion,
  a woman the wort thy that the hath thy mave sher bear in the the wire hat the wire the  in the what beare thot fart the  in the beart thou love the wire the wire thy thou hand in the when hat the what the wire the win

t the with so me the wire thy whet the  in the wir mine the fart the wirt and that thou my the whit thy that beart thy shat thy love the that bear bear beart the that love the  and mear bea
----- diversity: 0.5
----- Generating with seed: " master mistress of my passion,
  a woma"
 master mistress of my passion,
  a woman  art beall me wat meat so i sean in thy meart hine the srave wing the my hor  ter, beave,
  when that ghall han the heringe the mang the   ar wort i  the mor that noun hear my all whan thy tha

s fart ke that love,
  then beart and man thy beared and my love i  or bet en fing that mise the wile soon the whing the dat mit men in the wher bais all man timt des the berre,
                            
----- diversity: 1.0
----- Generating with seed: " master mistress of my passion,
  a woma"
 master mistress of my passion,
  a womasfeane i mear.


                                          n,
    mir ghat dimen thy  man sill.
  nim watr hasu as tand uvunde,
  walc

 wotmeiong floe helr swer blinc theev,
  whlo

s nor but fart beoil grors thy sowne pon kith the-dest apr whyt  oun sory gy,
  wid ise
  is  thow nolf ired ell mive thou beror diin,
   the wort,
, sents dead
  urr i nan and ron  of thour mirpt w ar try arf ollenke y
----- diversity: 1.2
----- Generating with seed: " master mistress of my passion,
  a woma"
 master mistress of my passion,
  a womaou hst elwirh ipe cove and merr,
  ifin grf last drich  trred or fown veite'ss srave,
  thet thee sears:
  fftstweve stadce, ond lrce' love pot that itr
 gorle,
  oreag- is

 yombmed wrovn
  thy thot bat tan ecdy met,
  weel of sevin shtemy besterp my foat.  
 alder firgtirt,
  f un
  is llust,
  aud ituuht thr eonr sin cavean hich favs thf shy wats ?
  awn fept,
  yavi i furtion, lover gait, fange,
Epoch 2/60
29184/34300 [========================>.....] - ETA: 4s - loss: 1.8216

...

hen respite,
  when i famfthed beauty and the say,
  when i sweet speck in their forture thee,
    that comess the world with thy heart all t
----- diversity: 0.5
----- Generating with seed: "ines to time thou grow'st,
    so long a"
ines to time thou grow'st,
    so long a can that thou bothen and for the willing thy will.


                     46  
  that be and thee thy sight is well may stell,
  when i found the bely dut 'st to beart,
  and shall not the speed me thee in suchy,
  when you show 

up a from ort) and with thou gove,
  that the faire have that his garped it make,
  when i sempy the would me behow,
  when i havows thy love, that in their fear,
    the
----- diversity: 1.0
----- Generating with seed: "ines to time thou grow'st,
    so long a"
ines to time thou grow'st,
    so long are thy thoublain, nor storn of blook,
  she sweet speed me the offecy art,
  sorr tow? y usking shall b a nother wrime,
  and my all thou behears and pladida and respent,
  wast nothfuch from his beain  themp

oin .


                     9each of the hears in your farth my can sead,
  to she, and rise live so bathy speme but grace,
  when to wind, but the look me winted wond,
  than ushytore make w
----- diversity: 1.2
----- Generating with seed: "ines to time thou grow'st,
    so long a"
ines to time thou grow'st,
    so long are phading the bravery vainterne.
  whom that she is mides to fenswalin benow,
  your suptery) the moth can deaothes.
  for but wert doth that thou calls, gen 
  a leave that his cheer,
  of s

hame splate nimmery dedied to kindward,
  whoich thy lended of hembeal  fauty take's lin,
  and true will, noin poment but birds whome by anreace,
  the uslieve take and heart tame, storn'st,
  when of memper
Epoch 16/60
32000/34300 [==========================>...] - ETA: 1s - loss: 1.0117

34300/34300 [==============================] - 28s 818us/step - loss: 1.0177

----- Generating text after Epoch: 15
----- diversity: 0.2
----- Generating with seed: "e with my moan.  
    receiving nought b"
e with my moan.  
    receiving nought by thy prause when i thee,
  that this here thy proud thee in this far from you,
  that the true, i destrine of the wind
  of this beauty th